<a href="https://colab.research.google.com/github/sakkarin31/PM2.5project/blob/main/miniMLmusic.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense
from tensorflow.keras.optimizers import Adam
from sklearn.preprocessing import StandardScaler
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.model_selection import train_test_split


In [6]:
# โหลด Dataset
df = pd.read_csv("tracks_features.csv")

# ดูตัวอย่างข้อมูล
df.head()

,id,name,album,album_id,artists,artist_ids,track_number,disc_number,explicit,danceability,...,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature,year,release_date
0,7lmeHLHBe4nmXzuXc0HDjk,Testify,The Battle Of Los Angeles,2eia0myWFgoHuttJytCxgX,['Rage Against The Machine'],['2d0hyoQ5ynDBnkvAbJKORj'],1,1,False,0.470,...,0.0727,0.02610,0.000011,0.3560,0.503,117.906,210133,4.0,1999,1999-11-02
1,1wsRitfRRtWyEapl0q22o8,Guerrilla Radio,The Battle Of Los Angeles,2eia0myWFgoHuttJytCxgX,['Rage Against The Machine'],['2d0hyoQ5ynDBnkvAbJKORj'],2,1,True,0.599,...,0.1880,0.01290,0.000071,0.1550,0.489,103.680,206200,4.0,1999,1999-11-02
2,1hR0fIFK2qRG3f3RF70pb7,Calm Like a Bomb,The Battle Of Los Angeles,2eia0myWFgoHuttJytCxgX,['Rage Against The Machine'],['2d0hyoQ5ynDBnkvAbJKORj'],3,1,False,0.315,...,0.4830,0.02340,0.000002,0.1220,0.370,149.749,298893,4.0,1999,1999-11-02
3,2lbASgTSoDO7MTuLAXlTW0,Mic Check,The Battle Of Los Angeles,2eia0myWFgoHuttJytCxgX,['Rage Against The Machine'],['2d0hyoQ5ynDBnkvAbJKORj'],4,1,True,0.440,...,0.2370,0.16300,0.000004,0.1210,0.574,96.752,213640,4.0,1999,1999-11-02
4,1MQTmpYOZ6fcMQc56Hdo7T,Sleep Now In the Fire,The Battle Of Los Angeles,2eia0myWFgoHuttJytCxgX,['Rage Against The Machine'],['2d0hyoQ5ynDBnkvAbJKORj'],5,1,False,0.426,...,0.0701,0.00162,0.105000,0.0789,0.539,127.059,205600,4.0,1999,1999-11-02


In [7]:
# เลือกเฉพาะ Feature ที่ใช้เทรนโมเดล
features = ['danceability', 'energy', 'key', 'loudness', 'speechiness',
            'acousticness', 'instrumentalness', 'valence', 'tempo']

# ลบค่าที่เป็น NaN
df_clean = df.dropna(subset=features)

# แปลงข้อมูลเป็น numpy array
X = df_clean[features].values

# Normalize ข้อมูล
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# แบ่งข้อมูล train/test
X_train, X_test = train_test_split(X_scaled, test_size=0.2, random_state=42)

print("ขนาดของข้อมูล Train:", X_train.shape)
print("ขนาดของข้อมูล Test:", X_test.shape)


ขนาดของข้อมูล Train: (963220, 9)
ขนาดของข้อมูล Test: (240805, 9)


In [8]:
# กำหนดขนาด Input
input_dim = X_train.shape[1]

# Encoder
input_layer = Input(shape=(input_dim,))
encoded = Dense(64, activation='relu')(input_layer)
encoded = Dense(32, activation='relu')(encoded)
encoded = Dense(16, activation='relu')(encoded)

# Decoder
decoded = Dense(32, activation='relu')(encoded)
decoded = Dense(64, activation='relu')(decoded)
decoded = Dense(input_dim, activation='linear')(decoded)

# สร้างโมเดล Autoencoder
autoencoder = Model(input_layer, decoded)

autoencoder.compile(optimizer=Adam(learning_rate=0.001), loss='mse')

# ดูโครงสร้างโมเดล
autoencoder.summary()


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)             │ (None, 9)                   │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 64)                  │             640 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 32)                  │           2,080 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ (None, 16)                  │             528 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_3 (Dense)                      │ (None, 32)                  │             544 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_4 (Dense)                      │ (None, 64)                  │           2,112 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_5 (Dense)                      │ (None, 9)                   │             585 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 6,489 (25.35 KB)

 Trainable params: 6,489 (25.35 KB)

 Non-trainable params: 0 (0.00 B)

In [9]:
# เทรนโมเดล
history = autoencoder.fit(X_train, X_train, epochs=50, batch_size=32, shuffle=True, validation_data=(X_test, X_test))



Epoch 1/50
30101/30101 ━━━━━━━━━━━━━━━━━━━━ 98s 3ms/step - loss: 0.0211 - val_loss: 4.8391e-05
Epoch 2/50
30101/30101 ━━━━━━━━━━━━━━━━━━━━ 93s 3ms/step - loss: 1.6245e-04 - val_loss: 4.8241e-05
Epoch 3/50
30101/30101 ━━━━━━━━━━━━━━━━━━━━ 92s 3ms/step - loss: 1.3341e-04 - val_loss: 7.9230e-05
Epoch 4/50
30101/30101 ━━━━━━━━━━━━━━━━━━━━ 94s 3ms/step - loss: 1.2513e-04 - val_loss: 6.2851e-04
Epoch 5/50
30101/30101 ━━━━━━━━━━━━━━━━━━━━ 92s 3ms/step - loss: 1.2234e-04 - val_loss: 7.8904e-05
Epoch 6/50
30101/30101 ━━━━━━━━━━━━━━━━━━━━ 88s 3ms/step - loss: 1.0716e-04 - val_loss: 1.9321e-05
Epoch 7/50
30101/30101 ━━━━━━━━━━━━━━━━━━━━ 146s 3ms/step - loss: 1.0384e-04 - val_loss: 3.2360e-05
Epoch 8/50
30101/30101 ━━━━━━━━━━━━━━━━━━━━ 86s 3ms/step - loss: 1.0125e-04 - val_loss: 1.6068e-05
Epoch 9/50
30101/30101 ━━━━━━━━━━━━━━━━━━━━ 94s 3ms/step - loss: 1.0019e-04 - val_loss: 5.0628e-05
Epoch 10/50
30101/30101 ━━━━━━━━━━━━━━━━━━━━ 141s 3ms/step - loss: 9.5347e-05 - val_loss: 3.8401e-05
Epoch 11/50

In [11]:
# สร้าง Encoder Model
encoder = Model(input_layer, encoded)

# แปลงเพลงเป็นเวกเตอร์
X_encoded = encoder.predict(X_scaled)

# เพิ่มคอลัมน์เวกเตอร์ลงใน DataFrame
df_clean["vector"] = list(X_encoded)

print("ตัวอย่างเวกเตอร์ของเพลงแรก:", df_clean["vector"].iloc[0])


37626/37626 ━━━━━━━━━━━━━━━━━━━━ 52s 1ms/step
ตัวอย่างเวกเตอร์ของเพลงแรก: [0.85804284 0.66906965 0.8085268  0.22608249 0.18567403 1.0780166
 0.5210808  0.         0.99111116 0.         0.38451597 0.73400885
 0.6572199  0.         0.6729716  1.743122  ]


In [12]:
# บันทึกโมเดล Autoencoder
autoencoder.save("autoencoder_model.h5")

# บันทึกโมเดล Encoder (เฉพาะส่วนที่ใช้แปลงเพลงเป็นเวกเตอร์)
encoder.save("encoder_model.h5")

# บันทึก StandardScaler ที่ใช้ Normalize ข้อมูล
import joblib
joblib.dump(scaler, "scaler.pkl")

print("✅ บันทึกโมเดลและตัวปรับมาตรฐานเรียบร้อยแล้ว!")


✅ บันทึกโมเดลและตัวปรับมาตรฐานเรียบร้อยแล้ว!


In [14]:
def recommend_similar(song_name, df, top_n=5):
    # ค้นหาเพลงใน DataFrame
    song = df[df["name"].str.lower() == song_name.lower()]

    if song.empty:
        return "ไม่พบเพลงนี้ใน Dataset"

    song_vector = np.array(song["vector"].iloc[0]).reshape(1, -1)

    # คำนวณความคล้ายคลึงระหว่างเพลงที่เลือกกับเพลงอื่น ๆ
    similarity = cosine_similarity(song_vector, np.stack(df["vector"]))

    indices = similarity.argsort()[0][-top_n-1:-1][::-1]

    # คืนค่าผลลัพธ์
    recommended_songs = df.iloc[indices][["name", "artists", "album"]]

    return recommended_songs

# ทดสอบแนะนำเพลง
recommend_similar("Testify", df_clean)

,name,artists,album
1167593,"Fixin' To Die (Life from Atlanta, 1/27/2012)",['Widespread Panic'],Wood (Live)
739000,Gonna Have A Party,['Alabama'],Livin' Lovin' Rockin' Rollin': The 25th Annive...
549958,If Love's the Answer,['Bruce Kulick'],Transformer
869326,Sleeping Somewhere Else,['The Slow Death'],Born Ugly Got Worse
904635,Smells Like Teen Spirit (with Joan Jett) - Live,"['Nirvana', 'Joan Jett & The Blackhearts']",Bad Reputation (Music from the Original Motion...


In [23]:
def recommend_similar(song_name, artist_name, df, top_n=5):
    # ค้นหาเพลงที่ชื่อเพลงตรงกัน และมีชื่อศิลปินอยู่ในลิสต์ artists
    song = df[(df["name"].str.lower() == song_name.lower()) &
              (df["artists"].apply(lambda x: artist_name.lower() in [a.lower() for a in eval(x)]))]

    if song.empty:
        return "ไม่พบเพลงนี้ใน Dataset"

    # ดึงเวกเตอร์ของเพลงที่ค้นหา
    song_vector = np.array(song["vector"].iloc[0]).reshape(1, -1)

    similarity = cosine_similarity(song_vector, np.stack(df["vector"]))

    indices = similarity.argsort()[0][::-1]

    recommended_songs = df.iloc[indices]

    # กรองเพลงที่เป็นเพลงต้นฉบับออก
    recommended_songs = recommended_songs[~((recommended_songs["name"].str.lower() == song_name.lower()) &
                                            (recommended_songs["artists"].apply(lambda x: artist_name.lower() in [a.lower() for a in eval(x)])))]

    # ลบเพลงซ้ำที่มีชื่อและศิลปินเหมือนกัน
    recommended_songs = recommended_songs.drop_duplicates(subset=["name", "artists"])

    recommended_songs = recommended_songs.head(top_n)

    return recommended_songs[["name", "artists", "album"]]

# ทดสอบแนะนำเพลง
recommend_similar("Will He", "Joji", df_clean)


,name,artists,album
791116,Citrus Light,"['Sin Fang', 'Sóley', 'Örvar Smárason']",Team Dreams
826612,The Sun Is Going Down II,['Sóley'],We Sink
443232,Green Eyes,['Steven Paul Ploog'],The Truth Is
416471,Trouble In Mind,['Jennifer Scott Quartet'],Live at the Cellar
485646,Meek; Wild,['Brooke Waggoner'],Go Easy Little Doves


In [ ]:
# # สร้าง Encoder Model
# encoder = Model(input_layer, encoded)

# # แปลงเพลงเป็นเวกเตอร์
# X_encoded = encoder.predict(X_scaled)

# from sklearn.metrics.pairwise import cosine_similarity

# # หาเพลงที่คล้ายกัน โดยใช้ชื่อเพลงและชื่อนักร้อง
# def recommend_similar(song_name, artist_name, df, top_n=5):
#     # ค้นหาเพลงและนักร้องที่ตรงกันใน DataFrame
#     song_index = df[(df['track_name'].str.lower() == song_name.lower()) &
#                     (df['artists'].str.lower() == artist_name.lower())].index

#     if len(song_index) == 0:
#         return "ไม่พบเพลงนี้ใน Dataset"

#     # ดึงเวกเตอร์ของเพลงที่ค้นหา
#     song_vector = X_encoded[song_index]

#     # คำนวณความคล้ายคลึงระหว่างเพลงที่เลือกและเพลงใน dataset
#     similarity = cosine_similarity(song_vector, X_encoded)

#     # หาเพลงที่คล้ายที่สุด (top_n เพลง) โดยใช้การจัดอันดับ
#     indices = similarity.argsort()[0][-top_n-1:-1][::-1]
#     recommended_songs = df.iloc[indices]

#     # กรองเพลงที่เหมือนกันทั้งชื่อเพลงและศิลปินออกจากผลลัพธ์
#     recommended_songs = recommended_songs[~((recommended_songs['track_name'].str.lower() == song_name.lower()) &
#                                             (recommended_songs['artists'].str.lower() == artist_name.lower()))]
#     #กรองซ้ำจากเพลงที่แนะนำแล้ว
#     recommended_songs = recommended_songs.drop_duplicates(subset=['track_name', 'artists'])
#     # ตรวจสอบว่าจำนวนเพลงที่แนะนำยังไม่ครบตามจำนวนที่ต้องการ
#     num_recommended = len(recommended_songs)

#     # เก็บชื่อเพลงและนักร้องที่แนะนำแล้ว
#     recommended_song_titles = set(recommended_songs['track_name'].str.lower() + " " + recommended_songs['artists'].str.lower())

#     # กรณีที่แนะนำเพลงยังไม่ครบจำนวน top_n
#     if num_recommended < top_n:
#         # หาเพลงที่คล้ายกันเพิ่มเติมจากที่เหลือ
#         remaining_indices = similarity.argsort()[0][::-1]
#         remaining_songs = df.iloc[remaining_indices]

#         # กรองเพลงที่ซ้ำ (ชื่อเพลงและนักร้องเดียวกัน) ออก
#         remaining_songs = remaining_songs[~((remaining_songs['track_name'].str.lower() == song_name.lower()) &
#                                             (remaining_songs['artists'].str.lower() == artist_name.lower()))]

#         # ลบเพลงที่เคยแนะนำไปแล้วจาก remaining_songs
#         remaining_songs = remaining_songs[~((remaining_songs['track_name'].str.lower() + " " + remaining_songs['artists'].str.lower()).isin(recommended_song_titles))]

#         # เอาผลลัพธ์ที่เหลือมารวมกันเพื่อให้ได้จำนวนที่ต้องการ
#         recommended_songs = pd.concat([recommended_songs, remaining_songs]).drop_duplicates(subset=['track_name', 'artists'])

#     # ตรวจสอบให้ได้จำนวนเพลงที่แนะนำครบตาม top_n
#     recommended_songs = recommended_songs.head(top_n)

#     # หากไม่พบเพลงที่คล้ายคลึงกัน (กรณีที่กรองแล้วไม่มีเพลงในรายการแนะนำ)
#     if recommended_songs.empty:
#         return "ไม่มีเพลงที่คล้ายคลึงกันใน Dataset"

#     return recommended_songs[['track_name', 'artists']]

# # ทดสอบแนะนำเพลง โดยใช้ชื่อเพลงและนักร้อง
# print(recommend_similar("Starboy", "The Weeknd;Daft Punk", df))


3563/3563 ━━━━━━━━━━━━━━━━━━━━ 5s 1ms/step
               track_name                                 artists
9314         Sem a Planta                       ConeCrewDiretoria
111438       Baseball Cap  Faithless;Rollo Armstrong;Sister Bliss
60119   Verified Choppa 2                                Marksman
111953      Lonely & Cold                                Gramatik
60678            Nice Suh                             Vybz Kartel
